In [1]:
!pip install django requests python-dotenv


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 43.3 MB/s eta 0:00:00


In [2]:
!django-admin startproject weatherproj
%cd weatherproj
!python manage.py startapp weather


/content/weatherproj


In [3]:
import os

os.environ["OWM_API_KEY"] = "d8a5220f96fecfe020396ab983335fb7"
os.environ["DEFAULT_CITY"] = "Peshawar"
os.environ["DEFAULT_UNITS"] = "metric"


In [4]:
from pathlib import Path
import os

BASE_DIR = Path.cwd()   # Use current working directory in Colab

SECRET_KEY = 'colab-secret-key'
DEBUG = True
ALLOWED_HOSTS = ['*']

INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'weather',
]

STATIC_URL = '/static/'
STATICFILES_DIRS = [BASE_DIR / 'weather' / 'static']

OWM_API_KEY = os.getenv("OWM_API_KEY", "d8a5220f96fecfe020396ab983335fb7")
OWM_BASE_URL = "https://api.openweathermap.org/data/2.5"
DEFAULT_CITY = os.getenv("DEFAULT_CITY", "Peshawar")
DEFAULT_UNITS = os.getenv("DEFAULT_UNITS", "metric")


In [5]:
# 1) Install dependencies
!pip install django requests python-dotenv pyngrok

# 2) Set environment variables (use your key)
import os
os.environ["OWM_API_KEY"] = "d8a5220f96fecfe020396ab983335fb7"
os.environ["DEFAULT_CITY"] = "Peshawar"
os.environ["DEFAULT_UNITS"] = "metric"

# 3) Create project + app
!django-admin startproject weatherproj
%cd weatherproj
!python manage.py startapp weather

# 4) Create folders for templates and static
import pathlib
pathlib.Path("weather/templates/weather/partials").mkdir(parents=True, exist_ok=True)
pathlib.Path("weather/static/css").mkdir(parents=True, exist_ok=True)


CommandError: '/content/weatherproj/weatherproj' already exists
/content/weatherproj/weatherproj
python3: can't open file '/content/weatherproj/weatherproj/manage.py': [Errno 2] No such file or directory


In [6]:
!pip install gradio requests matplotlib seaborn pandas


In [32]:
import gradio as gr
import requests

API_KEY = 'd8a5220f96fecfe020396ab983335fb7'

# -------------------- Functions --------------------

def get_current_weather(city):
    url = f'https://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}&units=metric'
    response = requests.get(url).json()

    if response.get('cod') != 200:
        return f"City not found: {city}"

    weather = response['weather'][0]['description']
    temp = response['main']['temp']
    humidity = response['main']['humidity']
    pressure = response['main']['pressure']
    wind = response['wind']['speed']

    result = (
        f"City: {city}\n"
        f"Weather: {weather}\n"
        f"Temperature: {temp} °C\n"
        f"Humidity: {humidity}%\n"
        f"Pressure: {pressure} hPa\n"
        f"Wind Speed: {wind} m/s"
    )
    return result

def get_air_quality(city):
    # Use OpenWeatherMap Air Pollution API
    # First, get city coordinates
    geo_url = f"http://api.openweathermap.org/geo/1.0/direct?q={city}&limit=1&appid={API_KEY}"
    geo_response = requests.get(geo_url).json()

    if len(geo_response) == 0:
        return f"City not found: {city}"

    lat = geo_response[0]['lat']
    lon = geo_response[0]['lon']

    aqi_url = f"http://api.openweathermap.org/data/2.5/air_pollution?lat={lat}&lon={lon}&appid={API_KEY}"
    aqi_response = requests.get(aqi_url).json()

    if 'list' not in aqi_response:
        return "Air quality data not available"

    aqi = aqi_response['list'][0]['main']['aqi']
    components = aqi_response['list'][0]['components']

    aqi_map = {1: "Good", 2: "Fair", 3: "Moderate", 4: "Poor", 5: "Very Poor"}

    result = (
        f"City: {city}\n"
        f"AQI: {aqi} ({aqi_map.get(aqi, 'Unknown')})\n"
        f"Components:\n"
        f"PM2.5: {components['pm2_5']} µg/m³\n"
        f"PM10: {components['pm10']} µg/m³\n"
        f"NO2: {components['no2']} µg/m³\n"
        f"O3: {components['o3']} µg/m³\n"
        f"CO: {components['co']} µg/m³"
    )
    return result

def get_hourly_forecast(city):
    url = f'https://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_KEY}&units=metric'
    response = requests.get(url).json()

    if response.get('cod') != "200":
        return f"City not found: {city}"

    forecast = response['list'][:8]  # Next 24 hours, 3-hour intervals
    result = f"Hourly Forecast for {city} (Next 24 hours):\n"
    for item in forecast:
        dt = item['dt_txt']
        temp = item['main']['temp']
        desc = item['weather'][0]['description']
        result += f"{dt}: {temp}°C, {desc}\n"
    return result

def get_5day_forecast(city):
    url = f'https://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_KEY}&units=metric'
    response = requests.get(url).json()

    if response.get('cod') != "200":
        return f"City not found: {city}"

    forecast = response['list']
    result = f"5-Day Forecast for {city}:\n"
    added_days = set()

    for item in forecast:
        date = item['dt_txt'].split(' ')[0]
        if date not in added_days:
            temp = item['main']['temp']
            desc = item['weather'][0]['description']
            result += f"{date}: {temp}°C, {desc}\n"
            added_days.add(date)
    return result

# -------------------- Gradio Interface --------------------

with gr.Blocks() as demo:
    gr.Markdown("## 🌤️EnviroSense AI")
    gr.Markdown(
        """
        <p style="font-family: 'Arial', sans-serif;
                  font-size: 16px;
                  color: #32CD32;  /* Light green */
                  font-weight: ;
                  text-align: justify;
                  line-height: 1.5;">
            EnviroSense AI is an AI-driven environmental intelligence platform that integrates multi-source sensor data,
            environmental indices, and machine-learning analytics to assess and predict environmental quality in real time.
            It provides automated sensing, intelligent insights, and risk guidance for researchers, industries, policymakers, and the public.
        </p>
        """,
        elem_id="description"
    )
    with gr.Row():
        city_input = gr.Textbox(label="Enter City", placeholder="e.g., Islamabad")

    with gr.Row():
        current_weather_btn = gr.Button("Current Weather")
        air_quality_btn = gr.Button("Air Quality")

    with gr.Row():
        hourly_forecast_btn = gr.Button("Hourly Forecast")
        five_day_forecast_btn = gr.Button("5-Day Forecast")

    output_box = gr.Textbox(label="Output", lines=15)

    current_weather_btn.click(get_current_weather, inputs=city_input, outputs=output_box)
    air_quality_btn.click(get_air_quality, inputs=city_input, outputs=output_box)
    hourly_forecast_btn.click(get_hourly_forecast, inputs=city_input, outputs=output_box)
    five_day_forecast_btn.click(get_5day_forecast, inputs=city_input, outputs=output_box)

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b015544a0852092c77.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
